In [1]:
import pandas as pd
import numpy as np

import requests
import csv
import json
import random
from bs4 import BeautifulSoup
from pprint import pprint
from fake_useragent import UserAgent, VERSION
import time
from datetime import datetime
from multiprocessing import Pool  #предоставляет возможность параллельных процессов

AREAS = ['moskva', 'moskovskaya_oblast']
ROOT_URL = 'https://auto.ru/'
ALL_MODELS_URL = 'https://auto.ru/htmlsitemap/mark_model_catalog.html'
CATALOG_URL = 'https://auto.ru/htmlsitemap/mark_model_catalog.html'
LISTING_URL = 'https://auto.ru/-/ajax/desktop/listing/'


## Парсинг данных с auto.ru через ajax-запрос с последующей обработкой ответа формата json.

In [126]:
# # Парсим список наименований моделей sitemap
# CATALOG_URL = 'https://auto.ru/htmlsitemap/mark_model_catalog.html'



# # def parse_models_list(url, brand):
# def parse_models_list(url):
#     """
#     Парсит названия моделей авто производителя
#     :param url: url
#     :param manufacturer_name: name of the manufacturer
#     :return: list of model names
#     """
#     response = requests.get(url)
#     soup = BeautifulSoup(response.text, 'html.parser')

#     sitemap_block = soup.find('div', class_='sitemap')
#     all_models_links = sitemap_block.find_all('a')
    
#     # Соберем все названия брендов
#     brands_lst = list(set([item['href'].split('/')[-3] for item in all_models_links]))
    
#     models_dict = {}
#     for brand_item in brands_lst:
#         models_dict[brand_item] = [item['href'].split('/')[-2] for item in all_models_links if brand_item in item.text.lower()]
#     # Сохраним модели только определенного производителя
#     # brand_models = [item['href'].split('/')[-2] for item in all_models_links if brand in item.text.lower()]
    
# #     return brand_models
#     return models_dict

In [127]:
# models_dct = parse_models_list(CATALOG_URL)

In [1]:
equipments_dict = {
    # Безопасность
    'airbag-driver': {'name': 'Подушка безопасности водителя', 'category': 'Безопасность'},
    'airbag-passenger': {'name': 'Подушка безопасности пассажира', 'category': 'Безопасность'},
    'airbag-side': {'name': 'Подушки безопасности боковые', 'category': 'Безопасность'},
    'airbag-rear-side': {'name': 'Подушки безопасности боковые задние', 'category': 'Безопасность'},
    'airbag-curtain': {'name': 'Подушки безопасности оконные (шторки)', 'category': 'Безопасность'},
    'knee-airbag': {'name': 'Подушки безопасности коленей водителя', 'category': 'Безопасность'},
    'isofix': {'name': 'Крепление детского кресла (задний ряд) ISOFIX', 'category': 'Безопасность'},
    'isofix-front': {'name': 'Крепление детского кресла (передний ряд) ISOFIX', 'category': 'Безопасность'},
    'asr': {'name': 'Антипробуксовочная система (ASR)', 'category': 'Безопасность'},
    'esp': {'name': 'Система стабилизации (ESP)', 'category': 'Безопасность'},
    'bas': {'name': 'Система помощи при торможении (BAS, EBD)', 'category': 'Безопасность'},
    'hsa': {'name': 'Система помощи при старте в гору (HSA)', 'category': 'Безопасность'},
    'abs': {'name': 'Антиблокировочная система (ABS)', 'category': 'Безопасность'},
    'glonass': {'name': 'ЭРА-ГЛОНАСС', 'category': 'Безопасность'},
    'drive-mode-sys': {'name': 'Система выбора режима движения', 'category': 'Безопасность'},
    'power-child-locks-rear-doors': {'name': 'Блокировка замков задних дверей', 'category': 'Безопасность'},
    'tyre-pressure': {'name': 'Датчик давления в шинах', 'category': 'Безопасность'},
    'drowsy-driver-alert-system': {'name': 'Датчик усталости водителя', 'category': 'Безопасность'},
    'laminated-safety-glass': {'name': 'Ламинированные боковые стекла', 'category': 'Безопасность'},
    'lane-keeping-assist': {'name': 'Система контроля за полосой движения', 'category': 'Безопасность'},
    'blind-spot': {'name': 'Система контроля слепых зон', 'category': 'Безопасность'},
    'night-vision': {'name': 'Система ночного видения', 'category': 'Безопасность'},
    'collision-prevention-assist': {'name': 'Система предотвращения столкновения', 'category': 'Безопасность'},
    'traffic-sign-recognition': {'name': 'Система распознавания дорожных знаков', 'category': 'Безопасность'},
    'vsm': {'name': 'Система стабилизации рулевого управления (VSM)', 'category': 'Безопасность'},
    'hdc': {'name': 'Система помощи при спуске', 'category': 'Безопасность'},
    
    # Мультимедиа 
    'aux': {'name': 'AUX', 'category': 'Мультимедиа'},
    'android-auto': {'name': 'Android Auto', 'category': 'Мультимедиа'},
    'bluetooth': {'name': 'Bluetooth', 'category': 'Мультимедиа'},
    'apple-carplay': {'name': 'CarPlay', 'category': 'Мультимедиа'},
    'usb': {'name': 'USB', 'category': 'Мультимедиа'},
    'audiopreparation': {'name': 'Аудиоподготовка', 'category': 'Мультимедиа'},
    'audiosystem-cd': {'name': 'Аудиосистема', 'category': 'Мультимедиа'},
    'music-super': {'name': 'Аудиосистема Hi-Fi', 'category': 'Мультимедиа'},
    'audiosystem-tv': {'name': 'Аудиосистема с TV', 'category': 'Мультимедиа'},
    'wireless-charger': {'name': 'Беспроводная зарядка для смартфона', 'category': 'Мультимедиа'},
    'voice-recognition': {'name': 'Голосовое управление', 'category': 'Мультимедиа'},
    'entertainment-system-for-rear-seat-passengers': {'name': 'Мультимедиа система для задних пассажиров', 'category': 'Мультимедиа'},
    'navigation': {'name': 'Навигационная система', 'category': 'Мультимедиа'},
    '12v-socket': {'name': 'Розетка 12V', 'category': 'Мультимедиа'},
    '220v-socket': {'name': 'Розетка 220V', 'category': 'Мультимедиа'},
    'ya-auto': {'name': 'Яндекс.Авто', 'category': 'Мультимедиа'},
    
    # Комфорт
    'auto-cruise': {'name': 'Адаптивный круиз-контроль', 'category': 'Комфорт'},
    'wheel-power': {'name': 'Активный усилитель руля', 'category': 'Комфорт'},
    'computer': {'name': 'Бортовой компьютер', 'category': 'Комфорт'},
    'remote-engine-start': {'name': 'Дистанционный запуск двигателя', 'category': 'Комфорт'},
    'power-latching-doors': {'name': 'Доводчик дверей', 'category': 'Комфорт'},
    'start-button': {'name': 'Запуск двигателя с кнопки', 'category': 'Комфорт'},
    '360-camera': {'name': 'Камера 360°', 'category': 'Комфорт'},
    'rear-camera': {'name': 'Камера задняя', 'category': 'Комфорт'},
    'front-camera': {'name': 'Камера передняя', 'category': 'comfort'},
    'climate-control-1': {'name': 'Климат-контроль 1-зонный', 'category': 'Комфорт'},
    'climate-control-2': {'name': 'Климат-контроль 2-зонный', 'category': 'Комфорт'},
    'multizone-climate-control': {'name': 'Климат-контроль многозонный', 'category': 'Комфорт'},
    'condition': {'name': 'Кондиционер', 'category': 'Комфорт'},
    'cruise-control': {'name': 'Круиз-контроль', 'category': 'Комфорт'},
    'multi-wheel': {'name': 'Мультифункциональное рулевое колесо', 'category': 'Комфорт'},
    'easy-trunk-opening': {'name': 'Открытие багажника без помощи рук', 'category': 'Комфорт'},
    'cooling-box': {'name': 'Охлаждаемый перчаточный ящик', 'category': 'Комфорт'},
    'park-assist-r': {'name': 'Парктроник задний', 'category': 'Комфорт'},
    'park-assist-f': {'name': 'Парктроник передний', 'category': 'Комфорт'},
    'steering-wheel-gear-shift-paddles': {'name': 'Подрулевые лепестки переключения передач', 'category': 'Комфорт'},
    'ashtray-and-cigarette-lighter': {'name': 'Прикуриватель и пепельница', 'category': 'Комфорт'},
    'programmed-block-heater': {'name': 'Программируемый предпусковой отопитель', 'category': 'Комфорт'},
    'projection-display': {'name': 'Проекционный дисплей', 'category': 'Комфорт'},
    'wheel-configuration1': {'name': 'Регулировка руля по вылету', 'category': 'Комфорт'},
    'wheel-configuration2': {'name': 'Регулировка руля по высоте', 'category': 'Комфорт'},
    'adj-pedals': {'name': 'Регулируемый педальный узел', 'category': 'Комфорт'},
    'wheel-memory': {'name': 'Рулевая колонка с памятью положения', 'category': 'Комфорт'},
    'start-stop-function': {'name': 'Система «старт-стоп»', 'category': 'Комфорт'},
    'auto-park': {'name': 'Система автоматической парковки', 'category': 'Комфорт'},
    'keyless-entry': {'name': 'Система доступа без ключа', 'category': 'Комфорт'},
    'wheel-power': {'name': 'Усилитель руля', 'category': 'Комфорт'},
    'electronic-gage-panel': {'name': 'Электронная приборная панель', 'category': 'Комфорт'},
    'electro-mirrors': {'name': 'Электропривод зеркал', 'category': 'Комфорт'},
    'electro-trunk': {'name': 'Электропривод крышки багажника', 'category': 'Комфорт'},
    'e-adjustment-wheel': {'name': 'Электрорегулировка руля', 'category': 'Комфорт'},
    'auto-mirrors': {'name': 'Электроскладывание зеркал', 'category': 'Комфорт'},
    'electro-window-back': {'name': 'Электростеклоподъёмники задние', 'category': 'Комфорт'},
    'electro-window-front': {'name': 'Электростеклоподъёмники передние', 'category': 'Комфорт'},
    
    # Салон
    'alcantara': {'name': 'Алькантара (Материал салона)', 'category': 'Салон'},
    'velvet-seats': {'name': 'Велюр (Материал салона)', 'category': 'Салон'},
    'rear-seat-heat-vent': {'name': 'Вентиляция задних сидений', 'category': 'Салон'},
    'front-seats-heat-vent': {'name': 'Вентиляция передних сидений', 'category': 'Салон'},
    'decorative-interior-lighting': {'name': 'Декоративная подсветка салона', 'category': 'Салон'},
    'sport-pedals': {'name': 'Декоративные накладки на педали', 'category': 'Салон'},
    'eco-leather': {'name': 'Искусственная кожа (Материал салона)', 'category': 'Салон'},
    'leather': {'name': 'Кожа (Материал салона)', 'category': 'Салон'},
    'seats-2': {'name': 'Количество мест: 2', 'category': 'Салон'},
    'seats-4': {'name': 'Количество мест: 4', 'category': 'Салон'},
    'seats-5': {'name': 'Количество мест: 5', 'category': 'Салон'},
    'seats-6': {'name': 'Количество мест: 6', 'category': 'Салон'},
    'seats-7': {'name': 'Количество мест: 7', 'category': 'Салон'},
    'combo-interior': {'name': 'Комбинированный (Материал салона)', 'category': 'Салон'},
    'servo': {'name': 'Люк', 'category': 'Салон'},
    'door-sill-panel': {'name': 'Накладки на пороги', 'category': 'Салон'},
    'wheel-heat': {'name': 'Обогрев рулевого колеса', 'category': 'Салон'},
    'wheel-leather': {'name': 'Отделка кожей рулевого колеса', 'category': 'Салон'},
    'leather-gear-stick': {'name': 'Отделка кожей рычага КПП', 'category': 'Салон'},
    'black-roof': {'name': 'Отделка потолка чёрной тканью', 'category': 'Салон'},
    'seat-memory': {'name': 'Память передних сидений', 'category': 'Салон'},
    'driver-seat-memory': {'name': 'Память сиденья водителя', 'category': 'Салон'},
    'panorama-roof': {'name': 'Панорамная крыша / лобовое стекло', 'category': 'Салон'},
    'front-centre-armrest': {'name': 'Передний центральный подлокотник', 'category': 'Салон'},
    'rear-seats-heat': {'name': 'Подогрев задних сидений', 'category': 'Салон'},
    'front-seats-heat': {'name': 'Подогрев передних сидений', 'category': 'Салон'},
    'passenger-seat-updown': {'name': 'Регулировка передних сидений по высоте', 'category': 'Салон'},
    'driver-seat-updown': {'name': 'Регулировка сиденья водителя по высоте', 'category': 'Салон'},
    'light-interior': {'name': 'Светлый салон', 'category': 'Салон'},
    'massage-seats': {'name': 'Сиденья с массажем', 'category': 'Салон'},
    # 'xxx': {'name': 'Складной столик на спинках передних сидений', 'category': 'Салон'},
    'folding-tables-rear': {'name': 'Складывающееся заднее сиденье', 'category': 'Салон'},
    'roller-blind-for-rear-window': {'name': 'Солнцезащитная шторка на заднем стекле', 'category': 'Салон'},
    'roller-blinds-for-rear-side-windows': {'name': 'Солнцезащитные шторки в задних дверях', 'category': 'Салон'},
    'sport-seats': {'name': 'Спортивные передние сиденья', 'category': 'Салон'},
    'fabric-seats': {'name': 'Ткань (Материал салона)', 'category': 'Салон'},
    'tinted-glass': {'name': 'Тонированные стекла', 'category': 'Салон'},
    'third-rear-headrest': {'name': 'Третий задний подголовник', 'category': 'Салон'},
    'third-row-seats': {'name': 'Третий ряд сидений', 'category': 'Салон'},
    'dark-interior': {'name': 'Тёмный салон', 'category': 'Салон'},
    'folding-front-passenger-seat': {'name': 'Функция складывания спинки сиденья пассажира', 'category': 'Салон'},
    'electro-rear-seat': {'name': 'Электрорегулировка задних сидений', 'category': 'Салон'},
    # 'xxx': {'name': 'Электрорегулировка передних сидений', 'category': 'Салон'},
    'driver-seat-electric': {'name': 'Электрорегулировка сиденья водителя', 'category': 'Салон'},
    
    # Защита от угона
    'volume-sensor': {'name': 'Датчик проникновения в салон (датчик объема)', 'category': 'Защита от угона'},
    'immo': {'name': 'Иммобилайзер', 'category': 'Защита от угона'},
    'alarm': {'name': 'Сигнализация', 'category': 'Защита от угона'},
    'feedback-alarm': {'name': 'Сигнализация с обратной связью', 'category': 'Защита от угона'},
    'lock': {'name': 'Центральный замок', 'category': 'Защита от угона'},
    
    # Обзор
    'automatic-lighting-control': {'name': 'Автоматический корректор фар', 'category': 'Обзор'},
    'rain-sensor': {'name': 'Датчик дождя', 'category': 'Обзор'},
    'light-sensor': {'name': 'Датчик света', 'category': 'Обзор'},
    'drl': {'name': 'Дневные ходовые огни', 'category': 'Обзор'},
    'xenon': {'name': 'Ксеноновые/Биксеноновые фары', 'category': 'Обзор'},
    'laser-lights': {'name': 'Лазерные фары', 'category': 'Обзор'},
    'light-cleaner': {'name': 'Омыватель фар', 'category': 'Обзор'},
    'ptf': {'name': 'Противотуманные фары', 'category': 'Обзор'},
    'led-lights': {'name': 'Светодиодные фары', 'category': 'Обзор'},
    'adaptive-light': {'name': 'Система адаптивного освещения', 'category': 'Обзор'},
    'high-beam-assist': {'name': 'Система управления дальним светом', 'category': 'Обзор'},
    'electro-mirrors': {'name': 'Электрообогрев боковых зеркал', 'category': 'Обзор'},
    'windscreen-heat': {'name': 'Электрообогрев зоны стеклоочистителей', 'category': 'Обзор'},
    'electro-window-front': {'name': 'Электрообогрев лобового стекла', 'category': 'Обзор'},
    'electro-window-back': {'name': 'Электрообогрев заднего стекла', 'category': 'Обзор'},
    'windcleaner-heat': {'name': 'Электрообогрев форсунок стеклоомывателей', 'category': 'Обзор'},
    
    #  Элементы экстерьера
    'airbrush': {'name': 'Аэрография', 'category': 'Элементы экстерьера'},
    'body-mouldings': {'name': 'Декоративные молдинги', 'category': 'Элементы экстерьера'},
    '14-inch-wheels': {'name': 'Диски 14', 'category': 'Элементы экстерьера'},
    '15-inch-wheels': {'name': 'Диски 15', 'category': 'Элементы экстерьера'},
    '16-inch-wheels': {'name': 'Диски 16', 'category': 'Элементы экстерьера'},
    '17-inch-wheels': {'name': 'Диски 17', 'category': 'Элементы экстерьера'},
    '18-inch-wheels': {'name': 'Диски 18', 'category': 'Элементы экстерьера'},
    '19-inch-wheels': {'name': 'Диски 19', 'category': 'Элементы экстерьера'},
    '20-inch-wheels': {'name': 'Диски 20', 'category': 'Элементы экстерьера'},
    '21-inch-wheels': {'name': 'Диски 21', 'category': 'Элементы экстерьера'},
    '22-inch-wheels': {'name': 'Диски 22', 'category': 'Элементы экстерьера'},
    'alloy-wheel-disks': {'name': 'Легкосплавные диски', 'category': 'Элементы экстерьера'},
    'paint-metallic': {'name': 'Металлик', 'category': 'Элементы экстерьера'},
    'body-kit': {'name': 'Обвес кузова', 'category': 'Элементы экстерьера'},
    'roof-rails': {'name': 'Рейлинги на крыше', 'category': 'Элементы экстерьера'},
    'steel-wheels': {'name': 'Стальные диски', 'category': 'Элементы экстерьера'},
    
    # Прочее
    'activ-suspension': {'name': 'Активная подвеска', 'category': 'Прочее'},
    # 'xxx': {'name': 'Газобаллонное оборудование', 'category': 'Прочее'},
    'reduce-spare-wheel': {'name': 'Докатка', 'category': 'Прочее'},
    # 'xxx': {'name': 'Защита картера', 'category': 'Прочее'},
    'air-suspension': {'name': 'Пневмоподвеска', 'category': 'Прочее'},
    'spare-wheel': {'name': 'Полноразмерное запасное колесо', 'category': 'Прочее'},
    'sport-suspension': {'name': 'Спортивная подвеска', 'category': 'Прочее'},
    # 'xxx': {'name': 'Фаркоп', 'category': 'Прочее'},
}


In [3]:
len(equipments_dict.keys())

160

In [129]:
# def parse_equipments(data):
#     """
#     Создает из переданных данные список словарей по категориям 
#     оборудования. 
#     :param data: a dicts
#     :return: a list of dicts 
#     """
#     if not data:
#         return None
    
#     # инициализируем словари
#     multimedia = {'name': 'Мультимедиа', 'values': []}
#     comfort = {'name': 'Комфорт', 'values': []}
#     salon = {'name': 'Салон', 'values': []}
#     safety = {'name': 'Безопасность', 'values': []}
#     other = {'name': 'Прочее', 'values': []}
#     overview = {'name': 'Обзор', 'values': []}
#     exterior = {'name': 'Элементы экстерьера', 'values': []}
#     theft_protection = {'name': 'Защита от угона', 'values': []}
    
    
#     # Проходимся по каждому ключу из
#     for key in data.keys():
#         if key in equipments_dict:
#             cat = equipments_dict[key]['category']
#             if cat == 'Мультимедиа':
#                 multimedia['values'].append(equipments_dict[key]['name'])
#             elif cat == 'Комфорт':
#                 comfort['values'].append(equipments_dict[key]['name'])
#             elif cat == 'Салон':
#                 salon['values'].append(equipments_dict[key]['name'])
#             elif cat == 'Безопасность':
#                 safety['values'].append(equipments_dict[key]['name'])
#             elif cat == 'Прочее':
#                 other['values'].append(equipments_dict[key]['name'])
#             elif cat == 'Обзор':
#                 overview['values'].append(equipments_dict[key]['name'])
#             elif cat == 'Элементы экстерьера':
#                 exterior['values'].append(equipments_dict[key]['name'])
#             elif cat == 'Защита от угона':
#                 theft_protection['values'].append(equipments_dict[key]['name'])
    
#     # prepare final list of dicts
#     cat_dicts = [multimedia, comfort, salon, safety, other, overview,
#                  exterior, theft_protection]
#     equipments = [tmp_dict for tmp_dict in cat_dicts if tmp_dict['values']]
    
#     return equipments
    

In [184]:
# def parse_offer(offer):
#     """
#     Парсит объявление и создает объект типа словарь с жесткой структурой
#     :param offer: dict like object
#     :return: fixed structure dictionary object
#     """
#     item_dict = {}
    
#     try:
#         item_dict['bodyType'] = offer['vehicle_info']['configuration']['human_name'].lower()
#     except:
#         item_dict['bodyType'] = None
        
#     try:
#         item_dict['brand'] = offer['vehicle_info']['mark_info']['code']
#     except:
#         item_dict['brand'] = None
        
#     try:    
#         item_dict['color_hex'] = offer['color_hex']
#     except:
#         item_dict['color_hex'] = None
        
#     try:
#         item_dict['description'] = offer['description']
#     except:
#         item_dict['description'] = None
        
#     try:
#         item_dict['engineDisplacement'] = offer['vehicle_info']['tech_param']['displacement']
#     except:
#         item_dict['engineDisplacement'] = None
        
#     try:
#         item_dict['enginePower'] = offer['vehicle_info']['tech_param']['power']
#     except:
#         item_dict['enginePower'] = None
        
#     try:
#         if offer['vehicle_info']['equipment'] != {}:
#             item_dict['equipment_dict'] = offer['vehicle_info']['equipment']
#         else:
#            item_dict['equipment_dict'] = None 
#     except:
#         item_dict['equipment_dict'] = None
        
        
        
#     try:
#         item_dict['model'] = offer['vehicle_info']['model_info']['code']
#     except:
#         item_dict['model'] = None

# # рабочий вариант, но требует дополнительной обработки значений признака
# #     try:
# #         item_dict['fuelType'] = offer['vehicle_info']['tech_param']['engine_type']
# #     except:
# #         item_dict['fuelType'] = None

#     try:
#         item_dict['fuelType'] = offer['vehicle_info']['tech_param']['engine_type']
#     except:
#         item_dict['fuelType'] = None
        
#     try:
#         item_dict['image'] = offer['state']['image_urls'][0]['sizes']['320x240']
#     except:
#         item_dict['image'] = None
        
#     try:
#         item_dict['mileage'] = offer['state']['mileage']
#     except:
#         item_dict['mileage'] = None

#     try:
#         item_dict['modelDate'] = offer['vehicle_info']['super_gen']['year_from']
#     except:
#         item_dict['modelDate'] = None
        
#     try:
#         item_dict['model_info'] = offer['vehicle_info']['model_info']
#     except:
#         item_dict['model_info'] = None
        
#     try:
#         item_dict['model_name'] = offer['vehicle_info']['model_info']['code']
#     except:
#         item_dict['model_name'] = None    
    
#     try:
#         item_dict['name'] = offer['vehicle_info']['tech_param']['human_name']
#     except:
#         item_dict['name'] = None
    
#     try:
#         item_dict['numberOfDoors'] = offer['vehicle_info']['configuration']['doors_count']
#     except:
#         item_dict['numberOfDoors'] = None
        
#     item_dict['parsing_unixtime'] = datetime.timestamp(datetime.today())   
        
#     try:
#         item_dict['priceCurrency'] = offer['price_info']['currency']
#     except:
#         item_dict['priceCurrency'] = None
        
#     try:
#         item_dict['productionDate'] = offer['documents']['year']
#     except:
#         item_dict['productionDate'] = None
        
#     try:
#         item_dict['sell_id'] = int(offer['id'])
#     except:
#         item_dict['sell_id'] = None
        
#     try:
#         item_dict['super_gen'] = offer['vehicle_info']['tech_param']
#     except:
#         item_dict['super_gen'] = None
        
#     try:
#         part_1 = offer['vehicle_info']['configuration']['body_type']
#         part_2 = offer['vehicle_info']['tech_param']['transmission']
# #         part_3 = str(float(round(int(offer['vehicle_info']['tech_param']['displacement'])/1000, 1)))
#         part_3 = str(offer['lk_summary'].split(' ')[0])
#         item_dict['vehicleConfiguration'] = f"{part_1} {part_2} {part_3}"
#     except:
#         item_dict['vehicleConfiguration'] = None       
        
#     try:
#         item_dict['vehicleTransmission'] = offer['vehicle_info']['tech_param']['transmission']
#     except:
#         item_dict['vehicleTransmission'] = None

#     try:
#         item_dict['vendor'] = offer['vehicle_info']['vendor']
#     except:
#         item_dict['vendor'] = None
        
#     try:
#         item_dict['Владельцы'] = round(float(offer['documents']['owners_number']))
#     except:
#         item_dict['Владельцы'] = None

#     if 'pts_original' in offer['documents']:
#         item_dict['ПТС'] = 'Оригинал' if offer['documents']['pts_original'] else 'Дубликат'
#     else:
#         item_dict['ПТС'] = None
        
#     try:
#         item_dict['Привод'] = offer['vehicle_info']['tech_param']['gear_type']
#     except:
#         item_dict['Привод'] = None
        
#     try:
#         item_dict['Руль'] = offer['vehicle_info']['steering_wheel']
#     except:
#         item_dict['Руль'] = None
        
#     try:
#         item_dict['Состояние'] = 'Не требует ремонта' if offer['state']['state_not_beaten'] else 'Битый / не на ходу'
#         # старая версия - удалить после отладки
#         #         item_dict['Состояние'] = 'Не требует ремонта' if 'state_not_beaten' in offer['state'] else 'Битый / не на ходу'
#     except:
#         item_dict['Состояние'] = None
    
#     try:
#         item_dict['Таможня'] = 'Растаможен' if offer['documents']['custom_cleared'] else 'Не растаможен'
#     except:
#         item_dict['Таможня'] = None
    
#     try:
#         item_dict['price'] = offer['price_info']['price']
#     except:
#         item_dict['price'] = None
        
#     try:
#         item_dict['price_timestamp'] = int(str(offer['price_history'][-1]['create_timestamp'])[:10])
#     except:
#         item_dict['price_timestamp'] = None
    
#     # Additional features
#     try:
#         item_dict['auto_class'] = offer['vehicle_info']['configuration']['auto_class']
#     except:
#         item_dict['auto_class'] = None
        
#     try:
#         item_dict['price_segment'] = offer['vehicle_info']['super_gen']['price_segment']
#     except:
#         item_dict['price_segment'] = None
        
#     try:
#         item_dict['seller_type'] = offer['seller_type']
#     except:
#         item_dict['seller_type'] = None
        

    

# # отказался парсить, ввиду большого кол-ва пропусков как в трейне и тесте    
# #     if 'purchase_date' in offer['documents']:
# #         item_dict['purchase_date'] = f"{offer['documents']['purchase_date']['month']}.{offer['documents']['purchase_date']['year']}"
# #     else:
# #         item_dict['purchase_date'] = None
        
# #  заменили на ключ equipment_dict
# #     try:
# #         item_dict['Комплектация'] = parse_equipments(offer['vehicle_info']['equipment'])
# #     except:
# #         item_dict['Комплектация'] = None
    
#     return item_dict
    

In [185]:
# def parse_offers_on_page(data):
#     """
#     Парсит данные с каждого объявления
#     :param data: dict like object
#     :return: list of offers
#     """
#     offers_on_page = []
#     for offer in data['offers']:
#         offers_on_page.append(parse_offer(offer))
    
#     return offers_on_page

In [186]:
# url = 'https://auto.ru/-/ajax/desktop/listing/'

# # параметры запроса по Москве и +200 км.
# # params = {
# #     "category":"cars",
# #     "section":"used",
# #     'damage_group': 'ANY',
# #     "output_type":"list",
# #     "top_days":"600",
# #     "page":1,
# #     'geo_radius':200,
# #     "page_size":50,
# #     "catalog_filter":[{
# #         "mark":"BMW",
# #         "model":"1ER"}],
# #     "geo_id":[213],
# # }

# # параметры запроса все авто + вся территория
# params = {
#     "category":"cars",
#     "section":"all",
#     'damage_group': 'ANY',
#     "output_type":"list",
#     "top_days":"600",
#     "page":1,
#     # 'geo_radius':200,
#     "page_size":50,
#     "catalog_filter":[{
#         "mark":"BMW",
#         "model":"1ER"}],
#     # "geo_id":[213],
# }

# headers = """
# Host: auto.ru
# User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; rv:80.0) Gecko/20100101 Firefox/80.0
# Accept: */*
# Accept-Language: ru-RU,ru;q=0.8,en-US;q=0.5,en;q=0.3
# Accept-Encoding: gzip, deflate, br
# Referer: https://auto.ru/moskovskaya_oblast/cars/bmw/1er/used/?output_type=list&page=2
# x-client-app-version: 202009.16.194507
# x-page-request-id: 849e1e014809b180dd6369ef515781b9
# x-client-date: 1600424297130
# x-csrf-token: 9619e4bde43b838e51fb6ec88f45abb5c1375dcf65e03e52
# x-requested-with: fetch
# content-type: application/json
# Origin: https://auto.ru
# Content-Length: 127
# Connection: keep-alive
# Cookie: _csrf_token=9619e4bde43b838e51fb6ec88f45abb5c1375dcf65e03e52; autoru_sid=a%3Ag5f6475822vadaim944uvk3m9k38df0q.ad1235e8bfb82d2e6ccc4e02279be7b8%7C1600419202951.604800.xvQtMDl8gSfu5EwSfkaggw.PAfJWMn3fHWWEPTK-mwweLB582QEQmHtFJ1FSi42Ji8; autoruuid=g5f6475822vadaim944uvk3m9k38df0q.ad1235e8bfb82d2e6ccc4e02279be7b8; suid=eb6987df5285415441449e94a1b41c80.d49f33fd60e906b91504804f403170d7; from_lifetime=1600424294237; from=direct; yuidcs=1; X-Vertis-DC=myt; yuidlt=1; yandexuid=4092999901600419204; counter_ga_all7=2; _ym_uid=1600419208166105885; _ym_d=1600424294; _ym_isad=2; crookie=tA8s7HCAVpgyzbZkV3K3hwGTs1rRKZRrq7yjQ6jXpUo/h4XStsR35QSw2azg6f2daVQSwtDViKNra0er/NkQPZYfcqM=; cmtchd=MTYwMDQxOTIwODYyMA==; _ga=GA1.2.464157848.1600422869; _gid=GA1.2.61052704.1600422869; _ym_wasSynced=%7B%22time%22%3A1600422869584%2C%22params%22%3A%7B%22eu%22%3A0%7D%2C%22bkParams%22%3A%7B%7D%7D; gdpr=0; _ym_visorc_22753222=b; cycada=fGsHIePtGydQvy2cOf9SC3d5qV8+cFZcW+Z4bWJWuno=; _ym_visorc_148422=w; _ym_visorc_148383=w"""
# headers = headers.strip().split('\n')

# dict_headers = {}
# for header in headers:
#     key, value = header.split(': ')
#     dict_headers[key] = value

    
# def collect_car_brand_ads(brand, brand_models):
#     """
#     Основная функция парсинга всех объявлений по продаже
#     моделей авто, переданного в параметре бренда.
#     :param brand: string
#     :return: list of dicts
#     """
#     start_time = datetime.now()  # tracking the time
#     brand_offers = []
#     # brand_models = parse_models_list(CATALOG_URL, brand)
#     # total_count = len(brand_models)

#     params['catalog_filter'][0]['mark'] = brand.upper()
#     for model in brand_models:
        
#         offers_by_model = []
#         # print(f"Model: {model}, осталось: {total_count}")
#         # total_count -= 1
#         params['catalog_filter'][0]['model'] = model.upper()
#         params['page'] = 1
        
#         try:
#             response = requests.post(url, json=params, headers=dict_headers)
#         except ConnectionError:
#             print(f"ConnectionError: {brand} {model}")
#             continue
#         except MaxRetryError:
#             print(f"MaxRetryError: {brand} {model}")
#             continue
#         except NewConnectionError:
#             print(f"NewConnectionError: {brand} {model}")
#             continue
            
#         try:
#             data = response.json()
#         except:
#             print(f"Raise error to read JSON obj by {brand} {model}.")
#             continue

#         if 'offers' not in data or len(data['offers']) == 0:  # объявлений нет
#             print(f"\n{model.upper()} - no offers.")
#             continue

#         # total_page_count
#         print(f"\nCollecting model: {model.upper()}...", end='')
#         total_page_count = int(data['pagination']['total_page_count'])

#         if total_page_count <= 1:
#             if len(data['offers']) == 0:
#                 continue
#             # offers_on_page = parse_offers_on_page(data)
#             offers_by_model += parse_offers_on_page(data)
#         else:
#             # Если страниц больше чем 1
#             for page in range(1, total_page_count + 1):
#                 # print(f"page: {page}")
#                 print(f".", end='')
#                 params['page'] = page
#                 response = requests.post(url, json=params, headers=dict_headers)
#                 data = response.json()
                
#                 if 'offers' not in data or len(data['offers']) == 0:  # дошли до конца списка предложений
#                     break
#                 # offers_on_page = parse_offers_on_page(data)
#                 offers_by_model += parse_offers_on_page(data)
#                 time.sleep(1)
#         print()
#         print(f"Collected: {len(offers_by_model)} offers.")
#         brand_offers += offers_by_model

#     print(f"{brand.upper()}: total collected {len(brand_offers)} offers.")
#     end_time = datetime.now()
#     print(f"Duration: {end_time - start_time}")
    
#     return brand_offers


## Парсим объявления по Европейским и Японским авто.


In [144]:
# РАБОЧИЙ КОД ПАРСИНГА - ГЛАВНАЯ ФУНКЦИЯ

# start = datetime.now()

# def get_models_dict():
#     models_dict = parse_models_list(CATALOG_URL)  # словарь "бренд": ["марка авто", "марка авто",]

#     # test_brands = ['SKODA', 'AUDI', 'HONDA', 'VOLVO', 'BMW', 'NISSAN', 'INFINITI',
#     #                'MERCEDES', 'TOYOTA', 'LEXUS', 'VOLKSWAGEN', 'MITSUBISHI']
#     european_brands = ['SKODA', 'PORSCHE', 'CITROEN', 'JAGUAR', 'RENAULT',
#                        'VOLKSWAGEN', 'VOLVO', 'MERCEDES', 'OPEL', 'BMW',
#                        'FIAT', 'MINI', 'AUDI', 'PEUGEOT']

#     japanese_brands = ['SUZUKI', 'HONDA', 'INFINITI', 'LEXUS', 'TOYOTA', 'SUBARU',
#                        'NISSAN', 'MITSUBISHI', 'MAZDA']

#     selected_brands = european_brands + japanese_brands

#     selected_brands = list(map(str.lower, selected_brands))  # перевели строки в нижний регистр

#     # all_models_dict.keys()

#     # Удалим из словаря те бренды, которых нет в тестовой выборке
#     items_to_delete = list(set(models_dict.keys()) - set(selected_brands))

#     for item in items_to_delete:
#         _ = models_dict.pop(item)
        
#     return models_dict
    
# # парсинг объявлений
# new_total_offers = []

# for brand, models in all_models_dict.items():
#     print(f"\nCurrent brand: {brand.upper()}")
#     new_total_offers += collect_car_brand_ads(brand, models)
#     time.sleep(10)
    
# print(f"Parsing done")
# print(f"Total offers count: {len(new_total_offers)}")

# end = datetime.now()
# delta = end - start
# print(f"Total time: {(end - start).seconds} sec.")


Current brand: MERCEDES

Collected: 29 offers.

Collected: 2 offers.

Collected: 95 offers.

Collected: 22 offers.

Collected: 23 offers.

Collected: 81 offers.

Collected: 596 offers.

Collected: 28 offers.

Collected: 3 offers.

Collected: 48 offers.

Collected: 7 offers.

Collected: 185 offers.

Collected: 16 offers.

Collected: 10 offers.

Collected: 40 offers.

CLK_KLASSE_AMG - no offers.

Collected: 104 offers.

Collected: 10 offers.

Collected: 1753 offers.

Collected: 47 offers.

Collected: 1 offers.

Collected: 259 offers.

Collected: 147 offers.

G_KLASSE_AMG_6X6 - no offers.

Collected: 268 offers.

Collected: 6 offers.

Collected: 285 offers.

Collected: 19 offers.

Collected: 270 offers.

GLB_AMG - no offers.

Collected: 617 offers.

Collected: 5 offers.

Collected: 237 offers.

Collected: 11 offers.

Collected: 991 offers.

Collected: 59 offers.

Collected: 445 offers.

Collected: 72 offers.

Collected: 129 offers.

Collected: 567 offers.

Collected: 15 offers.

Collecte


Collected: 6 offers.

N_BOX_SLASH - no offers.

N_ONE - no offers.

Collected: 3 offers.

N360 - no offers.

NSX - no offers.

Collected: 6 offers.

Collected: 1 offers.

Collected: 2 offers.

Collected: 1 offers.

PASSPORT - no offers.

Collected: 39 offers.

Collected: 17 offers.

QUINT - no offers.

Collected: 2 offers.

Collected: 2 offers.

S_MX - no offers.

Collected: 4 offers.

S500 - no offers.

S600 - no offers.

Collected: 1 offers.

Collected: 2 offers.

Collected: 11 offers.

Collected: 15 offers.

Collected: 11 offers.

STREET - no offers.

THATS - no offers.

TODAY - no offers.

Collected: 1 offers.

VAMOS - no offers.

Collected: 2 offers.

Collected: 1 offers.

Collected: 3 offers.

Collected: 3 offers.
HONDA: total collected 937 offers.
Duration: 0:00:44.849797

Current brand: TOYOTA

2000GT - no offers.

Collected: 23 offers.

Collected: 2 offers.

ALLION - no offers.

Collected: 24 offers.

Collected: 7 offers.

Collected: 6 offers.

Collected: 7 offers.

AURION - 


Collected: 13 offers.

VOLTZ - no offers.

Collected: 9 offers.

WILL - no offers.

Collected: 1 offers.

Collected: 4 offers.

Collected: 4 offers.

Collected: 44 offers.

Collected: 1 offers.
TOYOTA: total collected 3331 offers.
Duration: 0:02:20.529827

Current brand: MITSUBISHI

Collected: 1 offers.

500 - no offers.

Collected: 8 offers.

Collected: 3 offers.

Collected: 172 offers.

ATTRAGE - no offers.

BRABO - no offers.

Collected: 105 offers.

CELESTE - no offers.

CHALLENGER - no offers.

Collected: 12 offers.

Collected: 42 offers.

CORDIA - no offers.

DEBONAIR - no offers.

Collected: 11 offers.

Collected: 3 offers.

DELICA_D3 - no offers.

Collected: 3 offers.

Collected: 5 offers.

DIGNITY - no offers.

Collected: 8 offers.

Collected: 7 offers.

Collected: 23 offers.

Collected: 45 offers.

EK_ACTIVE - no offers.

EK_CLASSIC - no offers.

EK_CUSTOM - no offers.

EK_SPACE - no offers.

EK_SPORT - no offers.

Collected: 1 offers.

Collected: 1 offers.

Collected: 3 off


25 - no offers.

30 - no offers.

4 - no offers.

4CV - no offers.

Collected: 1 offers.

6 - no offers.

8 - no offers.

9 - no offers.

ALASKAN - no offers.

Collected: 193 offers.

Collected: 1 offers.

CAPTUR - no offers.

CARAVELLE - no offers.

Collected: 33 offers.

Collected: 4 offers.

CLIO_V6 - no offers.

DAUPHINE - no offers.

Collected: 14 offers.

Collected: 501 offers.

Collected: 14 offers.

FLORIDE - no offers.

Collected: 96 offers.

FREGATE - no offers.

FUEGO - no offers.

KADJAR - no offers.

Collected: 51 offers.

Collected: 322 offers.

Collected: 56 offers.

KWID - no offers.

Collected: 69 offers.

Collected: 12 offers.

LODGY - no offers.

Collected: 769 offers.

Collected: 252 offers.

Collected: 1 offers.

Collected: 2 offers.

RODEO - no offers.

Collected: 4 offers.

Collected: 535 offers.

SANDERO_RS - no offers.

Collected: 109 offers.

SPORT_SPYDER - no offers.

Collected: 75 offers.

TALISMAN - no offers.

Collected: 5 offers.

Collected: 2 offers.

C


Collected: 4 offers.

Collected: 2 offers.

X_90 - no offers.

XBEE - no offers.

XL_7 - no offers.
SUZUKI: total collected 901 offers.
Duration: 0:00:39.292981

Current brand: MINI

MINI - no offers.

MINI - no offers.

GEMINI - no offers.

FLAMINIA - no offers.

Collected: 3 offers.

Collected: 44 offers.

Collected: 173 offers.

Collected: 4 offers.

Collected: 201 offers.

Collected: 3 offers.

ROADSTER - no offers.

MINICA - no offers.

MINICAB - no offers.

PAJERO_MINI - no offers.

HYPERMINI - no offers.

PADMINI - no offers.

MINI - no offers.
MINI: total collected 428 offers.
Duration: 0:00:19.704677

Current brand: VOLVO

120_SERIES - no offers.

Collected: 1 offers.

164 - no offers.

Collected: 7 offers.

260 - no offers.

Collected: 2 offers.

Collected: 3 offers.

Collected: 11 offers.

480 - no offers.

66 - no offers.

Collected: 17 offers.

Collected: 4 offers.

780 - no offers.

Collected: 34 offers.

Collected: 25 offers.

Collected: 19 offers.

Collected: 28 offers


Collected: 1 offers.

CX - no offers.

Collected: 1 offers.

Collected: 8 offers.

Collected: 25 offers.

Collected: 6 offers.

DYANE - no offers.

E_MEHARI - no offers.

EVASION - no offers.

GS - no offers.

Collected: 9 offers.

LN - no offers.

NEMO - no offers.

SAXO - no offers.

SM - no offers.

Collected: 89 offers.

TRACTION_AVANT - no offers.

VISA - no offers.

Collected: 3 offers.

XM - no offers.

Collected: 6 offers.

Collected: 26 offers.

Collected: 1 offers.
CITROEN: total collected 991 offers.
Duration: 0:00:47.195592

Current brand: LEXUS

Collected: 7 offers.

Collected: 78 offers.

Collected: 78 offers.

GS_F - no offers.

Collected: 47 offers.

Collected: 1 offers.

Collected: 45 offers.

IS_F - no offers.

LC - no offers.

LFA - no offers.

Collected: 1 offers.

Collected: 69 offers.

Collected: 150 offers.

Collected: 67 offers.

RC - no offers.

RC_F - no offers.

Collected: 201 offers.

Collected: 4 offers.

Collected: 3 offers.
LEXUS: total collected 751 off

# Многопоточный парсер объявлений с auto.ru только Европейских и Японских авто

In [2]:
def parse_models_list(url):
    """
    Парсит названия моделей авто производителя
    :param url: url
    :param manufacturer_name: name of the manufacturer
    :return: list of model names
    """
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    sitemap_block = soup.find('div', class_='sitemap')
    all_models_links = sitemap_block.find_all('a')
    
    # Соберем все названия брендов
    brands_lst = list(set([item['href'].split('/')[-3] for item in \
                           all_models_links]))
    
    models_dict = {}
    for brand_item in brands_lst:
        models_dict[brand_item] = [item['href'].split('/')[-2] for item in \
                                   all_models_links if brand_item in \
                                   item.text.lower()]

    return models_dict

In [3]:
def parse_offer(offer):
    """
    Парсит объявление и создает объект типа словарь с жесткой структурой
    :param offer: dict like object
    :return: fixed structure dictionary object
    """
    item_dict = {}
    
    try:
        item_dict['bodyType'] = offer['vehicle_info']['configuration']['human_name'].lower()
    except:
        item_dict['bodyType'] = None
        
    try:
        item_dict['brand'] = offer['vehicle_info']['mark_info']['code']
    except:
        item_dict['brand'] = None
        
    try:    
        item_dict['color_hex'] = offer['color_hex']
    except:
        item_dict['color_hex'] = None
        
    try:
        item_dict['description'] = offer['description']
    except:
        item_dict['description'] = None
        
    try:
        item_dict['engineDisplacement'] = offer['vehicle_info']['tech_param']['displacement']
    except:
        item_dict['engineDisplacement'] = None
        
    try:
        item_dict['enginePower'] = offer['vehicle_info']['tech_param']['power']
    except:
        item_dict['enginePower'] = None
        
    try:
        if offer['vehicle_info']['equipment'] != {}:
            item_dict['equipment_dict'] = offer['vehicle_info']['equipment']
        else:
            item_dict['equipment_dict'] = None 
    except:
        item_dict['equipment_dict'] = None
        
    # Уже получаем признак model_info ниже 
#     try:
#         item_dict['model'] = offer['vehicle_info']['model_info']['code']
#     except:
#         item_dict['model'] = None

    try:
        item_dict['fuelType'] = offer['vehicle_info']['tech_param']['engine_type']
    except:
        item_dict['fuelType'] = None
        
    try:
        item_dict['image'] = offer['state']['image_urls'][0]['sizes']['320x240']
    except:
        item_dict['image'] = None
        
    try:
        item_dict['mileage'] = offer['state']['mileage']
    except:
        item_dict['mileage'] = None

    try:
        item_dict['modelDate'] = offer['vehicle_info']['super_gen']['year_from']
    except:
        item_dict['modelDate'] = None
        
    try:
        item_dict['model_info'] = offer['vehicle_info']['model_info']
    except:
        item_dict['model_info'] = None
        
    try:
        item_dict['model_name'] = offer['vehicle_info']['model_info']['code']
    except:
        item_dict['model_name'] = None    
    
    try:
        item_dict['name'] = offer['vehicle_info']['tech_param']['human_name']
    except:
        item_dict['name'] = None
    
    try:
        item_dict['numberOfDoors'] = offer['vehicle_info']['configuration']['doors_count']
    except:
        item_dict['numberOfDoors'] = None
        
    item_dict['parsing_unixtime'] = datetime.timestamp(datetime.today())   
        
    try:
        item_dict['priceCurrency'] = offer['price_info']['currency']
    except:
        item_dict['priceCurrency'] = None
        
    try:
        item_dict['productionDate'] = offer['documents']['year']
    except:
        item_dict['productionDate'] = None
        
    try:
        item_dict['sell_id'] = int(offer['id'])
    except:
        item_dict['sell_id'] = None
        
    try:
        item_dict['section'] = offer['section']
    except:
        item_dict['section'] = None
        
    # saleId
    try:
        item_dict['url_saleid'] = offer['saleId']
    except:
        item_dict['url_saleid'] = None
    
        
    try:
        item_dict['super_gen'] = offer['vehicle_info']['tech_param']
    except:
        item_dict['super_gen'] = None
        
    try:
        part_1 = offer['vehicle_info']['configuration']['body_type']
        part_2 = offer['vehicle_info']['tech_param']['transmission']
        part_3 = str(offer['lk_summary'].split(' ')[0])
        item_dict['vehicleConfiguration'] = f"{part_1} {part_2} {part_3}"
    except:
        item_dict['vehicleConfiguration'] = None       
        
    try:
        item_dict['vehicleTransmission'] = offer['vehicle_info']['tech_param']['transmission']
    except:
        item_dict['vehicleTransmission'] = None

    try:
        item_dict['vendor'] = offer['vehicle_info']['vendor']
    except:
        item_dict['vendor'] = None
        
    try:
        item_dict['Владельцы'] = round(float(offer['documents']['owners_number']))
    except:
        item_dict['Владельцы'] = None

    if 'pts' in offer['documents']:
        item_dict['ПТС'] = offer['documents']['pts']
    else:
        item_dict['ПТС'] = None
        
    try:
        item_dict['Привод'] = offer['vehicle_info']['tech_param']['gear_type']
    except:
        item_dict['Привод'] = None
        
    try:
        item_dict['Руль'] = offer['vehicle_info']['steering_wheel']
    except:
        item_dict['Руль'] = None
        
    try:
        item_dict['Состояние'] = 'Не требует ремонта' if offer['state']['state_not_beaten'] else 'Битый / не на ходу'
    except:
        item_dict['Состояние'] = None
    
    try:
        item_dict['Таможня'] = 'Растаможен' if offer['documents']['custom_cleared'] else 'Не растаможен'
    except:
        item_dict['Таможня'] = None
    
    try:
        item_dict['price'] = offer['price_info']['price']
    except:
        item_dict['price'] = None
        
#     try:
#         item_dict['price_timestamp'] = int(str(offer['price_history'][-1]['create_timestamp'])[:10])
#     except:
#         item_dict['price_timestamp'] = None
    
    try:
        item_dict['auto_class'] = offer['vehicle_info']['configuration']['auto_class']
    except:
        item_dict['auto_class'] = None
        
    try:
        item_dict['price_segment'] = offer['vehicle_info']['super_gen']['price_segment']
    except:
        item_dict['price_segment'] = None
        
    try:
        item_dict['seller_type'] = offer['seller_type']
    except:
        item_dict['seller_type'] = None
    
    return item_dict
    

In [4]:
def parse_offers_on_page(data):
    """
    Парсит данные с каждого объявления
    :param data: dict like object
    :return: list of offers
    """
    offers_on_page = []
    for offer in data['offers']:
        offers_on_page.append(parse_offer(offer))
    
    return offers_on_page

In [5]:
# параметры запроса все авто + вся территория
params = {
    "category":"cars",
    "section":"all",
    'damage_group': 'ANY',
    "output_type":"list",
    "top_days":"600",
    "page":1,
    "page_size":50,
    "catalog_filter":[{
        "mark":"BMW",
        "model":"1ER"}],
}

headers = """
Host: auto.ru
User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; rv:80.0) Gecko/20100101 Firefox/80.0
Accept: */*
Accept-Language: ru-RU,ru;q=0.8,en-US;q=0.5,en;q=0.3
Accept-Encoding: gzip, deflate, br
Referer: https://auto.ru/moskovskaya_oblast/cars/bmw/1er/used/?output_type=list&page=2
x-client-app-version: 202009.16.194507
x-page-request-id: 849e1e014809b180dd6369ef515781b9
x-client-date: 1600424297130
x-csrf-token: 9619e4bde43b838e51fb6ec88f45abb5c1375dcf65e03e52
x-requested-with: fetch
content-type: application/json
Origin: https://auto.ru
Content-Length: 127
Connection: keep-alive
Cookie: _csrf_token=9619e4bde43b838e51fb6ec88f45abb5c1375dcf65e03e52; autoru_sid=a%3Ag5f6475822vadaim944uvk3m9k38df0q.ad1235e8bfb82d2e6ccc4e02279be7b8%7C1600419202951.604800.xvQtMDl8gSfu5EwSfkaggw.PAfJWMn3fHWWEPTK-mwweLB582QEQmHtFJ1FSi42Ji8; autoruuid=g5f6475822vadaim944uvk3m9k38df0q.ad1235e8bfb82d2e6ccc4e02279be7b8; suid=eb6987df5285415441449e94a1b41c80.d49f33fd60e906b91504804f403170d7; from_lifetime=1600424294237; from=direct; yuidcs=1; X-Vertis-DC=myt; yuidlt=1; yandexuid=4092999901600419204; counter_ga_all7=2; _ym_uid=1600419208166105885; _ym_d=1600424294; _ym_isad=2; crookie=tA8s7HCAVpgyzbZkV3K3hwGTs1rRKZRrq7yjQ6jXpUo/h4XStsR35QSw2azg6f2daVQSwtDViKNra0er/NkQPZYfcqM=; cmtchd=MTYwMDQxOTIwODYyMA==; _ga=GA1.2.464157848.1600422869; _gid=GA1.2.61052704.1600422869; _ym_wasSynced=%7B%22time%22%3A1600422869584%2C%22params%22%3A%7B%22eu%22%3A0%7D%2C%22bkParams%22%3A%7B%7D%7D; gdpr=0; _ym_visorc_22753222=b; cycada=fGsHIePtGydQvy2cOf9SC3d5qV8+cFZcW+Z4bWJWuno=; _ym_visorc_148422=w; _ym_visorc_148383=w"""
headers = headers.strip().split('\n')

dict_headers = {}
for header in headers:
    key, value = header.split(': ')
    dict_headers[key] = value

    
def collect_car_brand_ads(brand, model, section):
    """
    Основная функция парсинга всех объявлений по продаже
    моделей авто, переданного в параметре бренда.
    :param brand: string
    :return: list of dicts
    """
    # brand_offers = []

    params['catalog_filter'][0]['mark'] = brand.upper()
        
    offers_by_model = []
    params['catalog_filter'][0]['model'] = model.upper()
    params['section'] = section
    params['page'] = 1

    try:
        response = requests.post(LISTING_URL, json=params,
                                 headers=dict_headers)
    except:
        print(f"Requests: raised exception by {brand} {model}")

    try:
        data = response.json()
    except:
        print(f"Raise error to read JSON obj by {brand} {model}.")
        return

    if 'offers' not in data or len(data['offers']) == 0:  # объявлений нет
        print(f"{brand.upper()} {model.upper()} {section}: no offers.")
        return 

    total_page_count = int(data['pagination']['total_page_count'])
    # print(f"TOTAL PAGE COUNT: {total_page_count} шт.")
#     if total_page_count <= 1:  # !!!!!!!!!!!!!!!!!!!!!!
#         if len(data['offers']) == 0:  # Удалить строчку, так как она есть выше
#             return
#         offers_by_model += parse_offers_on_page(data)
#     else:
    # Cтраниц 1 и больше
    for page in range(1, total_page_count + 1):
        # print(f".", end='')
        params['page'] = page
        response = requests.post(LISTING_URL, json=params,
                                 headers=dict_headers)
        data = response.json()

        if 'offers' not in data or len(data['offers']) == 0:  # дошли до конца списка
            break
        offers_by_model += parse_offers_on_page(data)
        time.sleep(random.randint(1, 3))

    # print(f"{brand} {model}: collected: {len(offers_by_model)} offers.")
    # brand_offers += offers_by_model

    print(f"{brand.upper()} {model.upper()} {section}: collected {len(offers_by_model)} offers.")
    
    return offers_by_model


In [6]:
def parse_models_dict():
    """
    Парсит наименования моделей по брендам.
    :return: a dict
    """
    models_dict = parse_models_list(CATALOG_URL)

    european_brands = ['SKODA', 'PORSCHE', 'CITROEN', 'JAGUAR', 'RENAULT',
                       'VOLKSWAGEN', 'VOLVO', 'MERCEDES', 'OPEL', 'BMW',
                       'FIAT', 'MINI', 'AUDI', 'PEUGEOT']

    japanese_brands = ['SUZUKI', 'HONDA', 'INFINITI', 'LEXUS', 'TOYOTA', 'SUBARU',
                       'NISSAN', 'MITSUBISHI', 'MAZDA']

    selected_brands = european_brands + japanese_brands

    # переведем строки в нижний регистр
    selected_brands = list(map(str.lower, selected_brands))

    # Удалим из словаря те бренды, которых нет в selected_brands
    items_to_delete = list(set(models_dict.keys()) - set(selected_brands))

    for item in items_to_delete:
        _ = models_dict.pop(item)
        
    return models_dict

def write_csv(data_list):
    """
    Получает список словарей и записывает
    в csv-файл
    :param data_list: a list of dicts
    """
    with open('cars_prices_multiprocessing_08_11_2020.csv', 'a') as csv_file:
        fieldnames = data_list[0].keys()
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        
        writer.writeheader()
        for item_dict in data_list:
            writer.writerow(item_dict)
            
    
    
    
def parse_model_offers(data):
    """
    Парсит все объявления по переданному
    бренду и модели авто. Записывает в csv-файлю.
    :param data: tuple of 2 strings
    :return: nothing
    """
    brand, model = data
    sections = ['new', 'used']
    for section in sections:
        data = collect_car_brand_ads(brand, model, section)
        if data:
            write_csv(data)

In [7]:
def get_cars_list():
    """
    Создает и возращает список кортежей (бренд, модель)
    :return: list of tuples
    """
    data_dict = parse_models_dict()
    tmp_lst = []
    
    for brand, models in data_dict.items():
        tmp_lst += [(brand, model) for model in models]
        
    return tmp_lst

In [8]:
def main():
    """
    Основная функция многопоточного парсера.
    """
    start = datetime.now()
    model_brand_tuples_lst = get_cars_list()
    
    with Pool(4) as p:           
       p.map(parse_model_offers, model_brand_tuples_lst)
    
    end = datetime.now()
    delta = end - start
    print(f"Total time: {round((end - start).seconds/60)} mins.")
    

In [449]:
main()

MERCEDES W201 new: no offers.TOYOTA CELICA new: no offers.

MERCEDES W191 new: no offers.
HONDA FR_V new: no offers.
MERCEDES W191 used: no offers.
MERCEDES W29 new: no offers.
MERCEDES W29 used: no offers.
MERCEDES X_KLASSE new: no offers.
MERCEDES W201 used: collected 25 offers.
TOYOTA CELICA used: collected 39 offers.
MERCEDES 190_SL new: no offers.
TOYOTA CELSIOR new: no offers.
TOYOTA CELSIOR used: no offers.
TOYOTA CENTURY new: no offers.
TOYOTA CENTURY used: no offers.
TOYOTA CHASER new: no offers.
MERCEDES X_KLASSE used: collected 2 offers.
OPEL ADAM new: no offers.
HONDA FR_V used: collected 2 offers.
OPEL ADAM used: no offers.
OPEL ADMIRAL new: no offers.HONDA FREED new: no offers.

OPEL ADMIRAL used: no offers.
OPEL AGILA new: no offers.
MERCEDES 190_SL used: collected 2 offers.
HONDA FREED used: collected 5 offers.
HONDA GRACE new: no offers.
OPEL AGILA used: collected 2 offers.
TOYOTA CHASER used: collected 25 offers.
HONDA GRACE used: no offers.
OPEL AMPERA new: no offers

MERCEDES EQC new: no offers.
TOYOTA PREVIA used: collected 11 offers.
MERCEDES EQC used: collected 2 offers.
TOYOTA PRIUS new: collected 1 offers.
HONDA ACCORD used: collected 202 offers.
HONDA ACTY new: no offers.
TOYOTA PRIUS used: collected 37 offers.
TOYOTA PRIUS_ALPHA new: no offers.
TOYOTA PRIUS_ALPHA used: collected 6 offers.MERCEDES G_KLASSE new: collected 87 offers.

HONDA ACTY used: collected 1 offers.
TOYOTA PRIUS_C new: no offers.
TOYOTA PRIUS_C used: no offers.
HONDA AIRWAVE new: no offers.
TOYOTA PRIUSPLUS new: no offers.
TOYOTA PRIUSPLUS used: collected 3 offers.
TOYOTA PROACE new: no offers.
TOYOTA PROACE used: no offers.
MITSUBISHI EK_SPACE new: no offers.
MITSUBISHI EK_SPACE used: no offers.
HONDA AIRWAVE used: collected 2 offers.
MITSUBISHI EK_SPORT new: no offers.
HONDA ASCOT new: no offers.
MITSUBISHI EK_SPORT used: no offers.
MITSUBISHI EK_WAGON new: no offers.
HONDA ASCOT used: collected 2 offers.
MITSUBISHI EK_WAGON used: collected 1 offers.
HONDA ASCOT_INNOVA n

NISSAN NAVARA used: collected 29 offers.
NISSAN NOTE new: no offers.
MITSUBISHI SPACE_STAR used: collected 26 offers.
MITSUBISHI SPACE_WAGON new: no offers.
NISSAN CEDRIC used: collected 7 offers.
MERCEDES GLE_KLASSE_AMG used: collected 17 offers.
NISSAN CEFIRO new: no offers.
NISSAN CEFIRO used: collected 14 offers.
MITSUBISHI SPACE_WAGON used: collected 20 offers.
MITSUBISHI STARION new: no offers.
NISSAN CHERRY new: no offers.
MITSUBISHI STARION used: no offers.
NISSAN CHERRY used: no offers.
MITSUBISHI STRADA new: no offers.
NISSAN CIMA new: no offers.
MITSUBISHI STRADA used: no offers.
NISSAN CIMA used: no offers.
NISSAN CLIPPER_RIO new: no offers.
NISSAN CLIPPER_RIO used: no offers.
MITSUBISHI TOPPO new: no offers.
NISSAN CREW new: no offers.
MITSUBISHI TOPPO used: no offers.
NISSAN CREW used: no offers.
NISSAN CUBE new: no offers.
MITSUBISHI TOWN_BOX new: no offers.
MITSUBISHI TOWN_BOX used: no offers.
MITSUBISHI TREDIA new: no offers.
MITSUBISHI TREDIA used: no offers.
MITSUBIS

NISSAN VANETTE new: no offers.
JAGUAR E_TYPE new: no offers.
JAGUAR E_TYPE used: no offers.
MERCEDES SIMPLEX used: collected 1 offers.
MERCEDES SL_KLASSE new: no offers.
RENAULT AVANTIME used: collected 1 offers.
NISSAN VANETTE used: collected 5 offers.
NISSAN VERSA new: no offers.RENAULT CAPTUR new: no offers.

RENAULT CAPTUR used: no offers.
NISSAN VERSA used: no offers.
RENAULT CARAVELLE new: no offers.
NISSAN WINGROAD new: no offers.
RENAULT CARAVELLE used: no offers.
RENAULT CLIO new: no offers.
MERCEDES SL_KLASSE used: collected 32 offers.
MERCEDES SL_KLASSE_AMG new: no offers.
JAGUAR F_PACE new: collected 42 offers.
MERCEDES SL_KLASSE_AMG used: collected 10 offers.
MERCEDES SLC_KLASSE new: no offers.
RENAULT CLIO used: collected 34 offers.
RENAULT CLIO_RS new: no offers.
NISSAN WINGROAD used: collected 10 offers.
MERCEDES SLC_KLASSE used: collected 1 offers.
MERCEDES SLC_KLASSE_AMG new: no offers.
MERCEDES SLC_KLASSE_AMG used: no offers.
MERCEDES SLK_KLASSE new: no offers.
JAGUA

AUDI A3 used: collected 184 offers.
BMW 5ER used: collected 938 offers.
INFINITI Q used: collected 2 offers.
BMW 501 new: no offers.
INFINITI Q30 new: no offers.
BMW 501 used: no offers.
BMW 502 new: no offers.
VOLVO S90 new: collected 23 offers.
BMW 502 used: no offers.
BMW 503 new: no offers.
BMW 503 used: no offers.
BMW 507 new: no offers.
BMW 507 used: no offers.
VOLVO S90 used: collected 18 offers.
VOLVO V40 new: no offers.
INFINITI Q30 used: collected 2 offers.
INFINITI Q40 new: no offers.
INFINITI Q40 used: no offers.
BMW 6ER new: collected 1 offers.
VOLVO V40 used: collected 12 offers.
VOLVO V40_CC new: no offers.
INFINITI Q50 new: collected 3 offers.
VOLVO V40_CC used: collected 20 offers.
INFINITI Q50 used: collected 37 offers.
VOLVO V50 new: no offers.
VOLVO V50 used: collected 6 offers.
VOLVO V60 new: no offers.
INFINITI Q60 new: collected 2 offers.
AUDI A4 new: collected 258 offers.
VOLVO V60 used: collected 3 offers.
BMW 6ER used: collected 110 offers.
BMW 600 new: no off

VOLKSWAGEN MULTIVAN used: collected 151 offers.
VOLKSWAGEN PARATI new: no offers.
VOLKSWAGEN PARATI used: no offers.
VOLKSWAGEN PASSAT new: collected 36 offers.
CITROEN C4 used: collected 299 offers.
CITROEN C4_AIRCROSS new: no offers.
SUZUKI GRAND_VITARA used: collected 266 offers.
SUZUKI HUSTLER new: no offers.
CITROEN C4_AIRCROSS used: collected 5 offers.
CITROEN C4_CACTUS new: no offers.
CITROEN C4_CACTUS used: no offers.
CITROEN C4_PICASSO new: no offers.
SUZUKI HUSTLER used: collected 3 offers.
SUZUKI IGNIS new: no offers.
SUZUKI IGNIS used: collected 15 offers.
CITROEN C4_PICASSO used: collected 71 offers.
CITROEN C4_SPACETOURER new: no offers.
CITROEN C4_SPACETOURER used: no offers.
CITROEN C5 new: no offers.
CITROEN C5 used: collected 94 offers.
CITROEN C5_AIRCROSS new: collected 32 offers.
SUZUKI JIMNY new: collected 173 offers.
CITROEN C5_AIRCROSS used: collected 1 offers.
CITROEN C6 new: no offers.
CITROEN C6 used: collected 1 offers.
CITROEN C8 new: no offers.
CITROEN C8 u

Так как парсинг проводился в несколько этапов, то произведем некоторые 
манипуляции с данными, до их слияния/сохранения для загрузки на Kaggle

In [63]:
df_1 = pd.read_csv('cars_prices_multiprocessing.csv')
df_1.shape

(162913, 35)

In [64]:
df_2 = pd.read_csv('cars_prices_multiprocessing_08_11_2020.csv')
df_2.shape

(56457, 36)

In [65]:
# В данные попадает аж 187 раз заголовоки для csv-таблицы. Удалим их
df_1 = df_1.loc[df_1.brand != 'brand']
display(df_1.shape)

df_2 = df_2.loc[df_2.brand != 'brand']
display(df_2.shape)

(161714, 35)

(55508, 36)

In [66]:
# Проверим наличие дубликатов по sell_id
display(df_1.sell_id.unique().shape)

display(df_2.sell_id.unique().shape)

(55884,)

(54256,)

In [67]:
# Удалим дубли по признаку sell_id
df_1.drop_duplicates(subset=['sell_id'], inplace=True)
display(df_1.shape)

df_2.drop_duplicates(subset=['sell_id'], inplace=True)
display(df_2.shape)

(55884, 35)

(54256, 36)

### Первичная обработка данных

При парсинге первого датасета df_1 мной была допущена ошика при получении значения ПТC, которая привела к тому, что значения Дубликат нет совсем, вместо него записано значение Оригинал.  
Стоит попробовать отдельно спарсить правильное значение этого признака, но перед этим сделаем быструю проверки есть ли какое либо влияние на стоимость авто значений этого признака.

In [454]:
df_2.ПТС.unique()

array(['ORIGINAL', 'DUPLICATE', nan], dtype=object)

In [455]:
df_2.isnull().sum()

bodyType                    2
brand                       0
color_hex                   0
description              1643
engineDisplacement          2
enginePower                 2
equipment_dict          12339
fuelType                    2
image                       0
mileage                     0
modelDate                   2
model_info                  0
model_name                  0
name                        2
numberOfDoors               2
parsing_unixtime            0
priceCurrency             215
productionDate              0
sell_id                     0
section                     0
url_saleid                  0
super_gen                   2
vehicleConfiguration        2
vehicleTransmission         2
vendor                      0
Владельцы               12799
ПТС                       333
Привод                      2
Руль                        0
Состояние                 886
Таможня                     0
price                     215
price_timestamp           215
auto_class

In [68]:
# удалим объекты с пропусками в след. признаках, а так же где нет цены, так как кол-во пропусков там минимально
cols_to_dropna = 'bodyType engineDisplacement enginePower fuelType modelDate name numberOfDoors super_gen vehicleConfiguration vehicleTransmission Привод price price_timestamp price_segment'.split(' ')


df_1 = df_1.dropna(subset=cols_to_dropna)
df_2 = df_2.dropna(subset=cols_to_dropna)

In [69]:
df_1.price = df_1.price.astype('int64')
df_1.sell_id = df_1.sell_id.astype('int64')

df_2.price = df_2.price.astype('int64')
df_2.sell_id = df_2.sell_id.astype('int64')

In [20]:
df_2[['ПТС', 'price']].groupby('ПТС').median()

,price
ПТС,
DUPLICATE,420000
ORIGINAL,1050000


Значение признака price меняется в 2.5 раза в зависимости значения признака ПТС, поэтому признак ПТС является значимым и его необходимо актуализировать.  
К сожалению, ввиду 

## Удаление не корректных данных по  ПТС и конкатенация двух трейнов

In [70]:
def convert_to_datetime(t_stamps):
    """
    Функция получает массив временных меток и
    преобразует каждый элемент к формату datetime
    :param t_stamps: a list of unix time stamps
    :return a list of datetime objects
    """
    
    return [datetime.fromtimestamp(t_stamp) for t_stamp in t_stamps.astype('float64')]





test = pd.read_csv('test_26_10_2020.csv', parse_dates=['parsing_unixtime'],
                    date_parser=convert_to_datetime)

test.shape

(34686, 32)

In [71]:
###
# Посмотрим какое кол-во объявлений из трейна есть в тесте.  Это нужно для того, чтобы постараться по максимуму свосстановить корректные значения признака ПТС в трейне из теста по признаку sel_id

train_1_ids = df_1.sell_id.unique()
train_2_ids = df_2.sell_id.unique()
test_ids = test.sell_id.unique()

diff_train_1_train_2_ids = set(train_1_ids) - set(train_2_ids)
common_tr_1_tr_2_and_test_ids = set(diff_train_1_train_2_ids) & set(test_ids)

print(f"В тесте есть: {len(common_tr_1_tr_2_and_test_ids)} записей")

# Сокращаем df_1 до тех записей, которые есть в common_tr_1_tr_2_and_test_ids
df_1 = df_1[df_1.sell_id.isin(common_tr_1_tr_2_and_test_ids)]
df_1.shape

В тесте есть: 3669 записей


(3669, 35)

In [72]:
# Теперь пройдемся по каждой строке df_1 и заменим значение признака ПТС
test_ids = test.sell_id.unique()

def fill_pts_value_from_test(row):
    curr_sell_id = row['sell_id']
    
    if curr_sell_id in test_ids:
        pts_in_test = test[test.sell_id == curr_sell_id].ПТС.iloc[0]
        row['ПТС'] = pts_in_test
    
    return row

df_1 = df_1.apply(fill_pts_value_from_test, axis=1)

In [75]:
# ОТЛИЧНО - ЗАМЕНИЛИ ЗНАЧЕНИЯ НА ТЕ, КОТОРЫЕ ЕСТЬ В ТЕСТЕ.

df_1.ПТС.value_counts()

Оригинал    3247
Дубликат     422
Name: ПТС, dtype: int64

### Объединение df_1 и df_2

In [79]:
# для df_1
cols_to_del_df_1 = ['model', 'price_timestamp']
cols_to_add_df_1 = ['section', 'url_saleid']

df_1.drop(labels=cols_to_del_df_1, axis=1, inplace=True)
for col in cols_to_add_df_1:
    df_1[col] = np.nan

# для df_2
cols_to_del_df_2 = ['price_timestamp']
df_2.drop(labels=cols_to_del_df_2, axis=1, inplace=True)

In [80]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3669 entries, 18 to 162894
Data columns (total 35 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   bodyType              3669 non-null   object 
 1   brand                 3669 non-null   object 
 2   color_hex             3669 non-null   object 
 3   description           3479 non-null   object 
 4   engineDisplacement    3669 non-null   object 
 5   enginePower           3669 non-null   object 
 6   equipment_dict        2728 non-null   object 
 7   fuelType              3669 non-null   object 
 8   image                 3669 non-null   object 
 9   mileage               3669 non-null   object 
 10  modelDate             3669 non-null   object 
 11  model_info            3669 non-null   object 
 12  model_name            3669 non-null   object 
 13  name                  3669 non-null   object 
 14  numberOfDoors         3669 non-null   object 
 15  parsing_unixtime  

In [81]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 54039 entries, 0 to 56456
Data columns (total 35 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   bodyType              54039 non-null  object
 1   brand                 54039 non-null  object
 2   color_hex             54039 non-null  object
 3   description           52410 non-null  object
 4   engineDisplacement    54039 non-null  object
 5   enginePower           54039 non-null  object
 6   equipment_dict        41780 non-null  object
 7   fuelType              54039 non-null  object
 8   image                 54039 non-null  object
 9   mileage               54039 non-null  object
 10  modelDate             54039 non-null  object
 11  model_info            54039 non-null  object
 12  model_name            54039 non-null  object
 13  name                  54039 non-null  object
 14  numberOfDoors         54039 non-null  object
 15  parsing_unixtime      54039 non-null

In [83]:
df = df_1.append(df_2, sort=False).reset_index(drop=True)
df.shape

(57708, 35)

In [86]:
# Write to csv-file
df.to_csv('car_price_multiprocessing_full_dataset_08_11_2020.csv', index=False, encoding='utf-8')

## Read the parsed data

In [88]:
df = pd.read_csv('car_price_multiprocessing_full_dataset_08_11_2020.csv')

In [91]:
df.tail()

,bodyType,brand,color_hex,description,engineDisplacement,enginePower,equipment_dict,fuelType,image,mileage,...,Привод,Руль,Состояние,Таможня,price,auto_class,price_segment,seller_type,section,url_saleid
57703,седан,VOLKSWAGEN,040001,"Машина в хорошем состоянии для своих лет,при о...",1984,115,NaN,GASOLINE,//avatars.mds.yandex.net/get-autoru-vos/219862...,260000,...,FORWARD_CONTROL,LEFT,NaN,Растаможен,40000,C,MEDIUM,PRIVATE,used,1092168300-c499de7e
57704,седан,VOLKSWAGEN,22A0F8,"Полностью обслужена,все дно переварили в июле,...",1984,115,"{'passenger-seat-updown': True, 'airbag-passen...",GASOLINE,//avatars.mds.yandex.net/get-autoru-vos/218118...,250000,...,FORWARD_CONTROL,LEFT,Не требует ремонта,Растаможен,100000,C,MEDIUM,PRIVATE,used,1100541674-b598d47f
57705,седан,VOLKSWAGEN,97948F,Покупалась временно до покупки нового Авто. Ст...,1781,90,NaN,GASOLINE,//avatars.mds.yandex.net/get-autoru-vos/220317...,180580,...,FORWARD_CONTROL,LEFT,Не требует ремонта,Растаможен,65000,C,MEDIUM,PRIVATE,used,1096783262-25c15bab
57706,седан,VOLKSWAGEN,EE1D19,"В хорошем состоянии, сел и поехал. Масло мен...",1781,90,"{'engine-proof': True, 'ptf': True, 'airbag-dr...",GASOLINE,//avatars.mds.yandex.net/get-autoru-vos/215936...,177000,...,FORWARD_CONTROL,LEFT,Не требует ремонта,Растаможен,110000,C,MEDIUM,PRIVATE,used,1099086896-a15245c2
57707,седан,VOLKSWAGEN,0000CC,NaN,1595,75,"{'alloy-wheel-disks': True, 'tinted-glass': Tr...",GASOLINE,//avatars.mds.yandex.net/get-autoru-vos/217450...,320000,...,FORWARD_CONTROL,LEFT,Не требует ремонта,Растаможен,100000,C,MEDIUM,PRIVATE,used,1101194108-28aff187


### Произведем первичные преобразования значений в признаках:
- fuelType
- color - создадим, а color_hex удалим
- Руль
- Привод

In [92]:
# преобразуем значения признака fuelType
fuel_type_converter = {
    'GASOLINE': 'бензин',
    'DIESEL': 'дизель',
    'HYBRID': 'гибрид',
    'ELECTRO': 'электро',
    'LPG': 'газ',
}

color_convertor = {
    '040001': 'черный',
    'FAFBFB': 'белый',
    '0000CC': 'синий',
    '97948F': 'серый',
    'CACECB': 'серебристый',
    '200204': 'коричневый',
    'EE1D19': 'красный',
    '007F00': 'зелёный',
    '22A0F8': 'голубой',
    'C49648': 'бежевый.',
    'DEA522': 'золотистый',
    'FF8649': 'оранжевый',
    '660099': 'фиолетовый',
    '4A2197': 'пурпурный',
    'FFD600': 'жёлтый',
    'FFC0CB': 'розовый',    
}

#  Нужно применить
wheel_convertor = {
    'LEFT': 'левый',
    'RIGHT': 'правый',
}

transmission_converter = {
    'MECHANICAL': 'механическая',
    'AUTOMATIC': 'автоматическая',
    'ROBOT': 'роботизированная',
    'VARIATOR': 'вариаторная'
}

gear_type_convertor = {
    'REAR_DRIVE': 'задний',
    'ALL_WHEEL_DRIVE': 'полный',
    'FORWARD_CONTROL': 'передний',
}

pts_convertor = {
    'ORIGINAL': 'Оригинал',
    'DUPLICATE': 'Дубликат',
}

def convert_features(data):
    data.fuelType.replace(fuel_type_converter, inplace=True)

    # преобразуем значения цветов из hex-формата в строчное название цвета
    data['color'] = data.color_hex.apply(lambda x: color_convertor[x])
    data.drop(labels='color_hex', axis=1, inplace=True)
    
    # Преобразуем признак steering_wheel
    data['Руль'].replace(wheel_convertor, inplace=True)
    
    # Преобразуем признак vehicleTransmission
    data.vehicleTransmission.replace(transmission_converter, inplace=True)
    
    # Преобразуем признак gear_type
    data['Привод'].replace(gear_type_convertor, inplace=True)
    
    # Преобразуем признак ПТС
    data['ПТС'].replace(pts_convertor, inplace=True)
    
    # Переведем в верхний регистра название бренда
    data.brand = data.brand.str.upper()
    
    return data

In [93]:
# df = convert_features(mult_df)
df = convert_features(df)

In [95]:
df.isnull().sum()

bodyType                    0
brand                       0
description              1819
engineDisplacement          0
enginePower                 0
equipment_dict          13200
fuelType                    0
image                       0
mileage                     0
modelDate                   0
model_info                  0
model_name                  0
name                        0
numberOfDoors               0
parsing_unixtime            0
priceCurrency               0
productionDate              0
sell_id                     0
super_gen                   0
vehicleConfiguration        0
vehicleTransmission         0
vendor                      0
Владельцы               12799
ПТС                       333
Привод                      0
Руль                        0
Состояние                 880
Таможня                     0
price                       0
auto_class                312
price_segment               0
seller_type                 0
section                  3669
url_saleid

In [325]:
# удалим объекты с пропусками в след. признаках, а так же где нет цены, так как кол-во пропусков там минимально
cols_to_dropna = 'bodyType engineDisplacement enginePower fuelType modelDate name numberOfDoors super_gen vehicleConfiguration vehicleTransmission Привод price price_timestamp price_segment'.split(' ')

df = df.dropna(subset=cols_to_dropna)

### Запишем получившийся датасет в csv-файл

In [96]:
df.to_csv('car_price_08_11_2020_final.csv', index=False, encoding='utf-8')

In [7]:
import numpy as np

In [11]:
np.linspace(start = 100, stop = 1500, num = 50)

array([ 100.        ,  128.57142857,  157.14285714,  185.71428571,
        214.28571429,  242.85714286,  271.42857143,  300.        ,
        328.57142857,  357.14285714,  385.71428571,  414.28571429,
        442.85714286,  471.42857143,  500.        ,  528.57142857,
        557.14285714,  585.71428571,  614.28571429,  642.85714286,
        671.42857143,  700.        ,  728.57142857,  757.14285714,
        785.71428571,  814.28571429,  842.85714286,  871.42857143,
        900.        ,  928.57142857,  957.14285714,  985.71428571,
       1014.28571429, 1042.85714286, 1071.42857143, 1100.        ,
       1128.57142857, 1157.14285714, 1185.71428571, 1214.28571429,
       1242.85714286, 1271.42857143, 1300.        , 1328.57142857,
       1357.14285714, 1385.71428571, 1414.28571429, 1442.85714286,
       1471.42857143, 1500.        ])